In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_csv('../../../data/Clean_data.csv')

In [3]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Seattle,Snoqualmie,Woodinville,sqft_living_diff,sqft_lot_diff,year,x0_fall,x0_spring,x0_summer,x0_winter
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,-160.0,0.0,2014.0,1.0,0.0,0.0,0.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,880.0,-397.0,2014.0,0.0,0.0,0.0,1.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0.0,3.0,6.0,...,0.0,0.0,0.0,-1950.0,1938.0,2015.0,0.0,0.0,0.0,1.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0.0,5.0,7.0,...,1.0,0.0,0.0,600.0,0.0,2014.0,0.0,0.0,0.0,1.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0.0,3.0,8.0,...,0.0,0.0,0.0,-120.0,577.0,2015.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21591,360000.0,3.0,2.50,1530.0,1131.0,3.0,0,0.0,3.0,8.0,...,1.0,0.0,0.0,0.0,-378.0,2014.0,NaN,NaN,NaN,NaN
21592,400000.0,4.0,2.50,2310.0,5813.0,2.0,0,0.0,3.0,8.0,...,1.0,0.0,0.0,480.0,-1387.0,2015.0,NaN,NaN,NaN,NaN
21593,402101.0,2.0,0.75,1020.0,1350.0,2.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,0.0,-657.0,2014.0,NaN,NaN,NaN,NaN
21594,400000.0,3.0,2.50,1600.0,2388.0,2.0,0,0.0,3.0,8.0,...,0.0,0.0,0.0,190.0,1101.0,2015.0,NaN,NaN,NaN,NaN


In [4]:
df.drop(['zipcode'], axis=1, inplace=True)

In [5]:
df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21242 entries, 0 to 21418
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             21242 non-null  float64
 1   bedrooms          21242 non-null  float64
 2   bathrooms         21242 non-null  float64
 3   sqft_living       21242 non-null  float64
 4   sqft_lot          21242 non-null  float64
 5   floors            21242 non-null  float64
 6   waterfront        21242 non-null  int64  
 7   view              21242 non-null  float64
 8   condition         21242 non-null  float64
 9   grade             21242 non-null  float64
 10  sqft_above        21242 non-null  float64
 11  basement          21242 non-null  int64  
 12  yr_built          21242 non-null  float64
 13  yr_renovated      21242 non-null  float64
 14  lat               21242 non-null  float64
 15  long              21242 non-null  float64
 16  sqft_living15     21242 non-null  float6

In [7]:
mid_grades = df.loc[(df['grade'] == 7) | (df['grade'] == 8) | (df['grade'] == 9)]

In [8]:
mid_grades

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Seattle,Snoqualmie,Woodinville,sqft_living_diff,sqft_lot_diff,year,x0_fall,x0_spring,x0_summer,x0_winter
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,-160.0,0.0,2014.0,1.0,0.0,0.0,0.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,880.0,-397.0,2014.0,0.0,0.0,0.0,1.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0.0,5.0,7.0,...,1.0,0.0,0.0,600.0,0.0,2014.0,0.0,0.0,0.0,1.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0.0,3.0,8.0,...,0.0,0.0,0.0,-120.0,577.0,2015.0,0.0,0.0,0.0,1.0
6,257500.0,3.0,2.25,1715.0,6819.0,2.0,0,0.0,3.0,7.0,...,0.0,0.0,0.0,-523.0,0.0,2014.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21414,700000.0,3.0,2.50,2230.0,4006.0,2.0,0,0.0,3.0,8.0,...,1.0,0.0,0.0,0.0,-174.0,2015.0,0.0,1.0,0.0,0.0
21415,800000.0,4.0,3.25,3540.0,159430.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,0.0,1600.0,-232610.0,2014.0,0.0,0.0,0.0,1.0
21416,899000.0,4.0,3.50,2490.0,5500.0,2.0,0,0.0,3.0,9.0,...,1.0,0.0,0.0,780.0,0.0,2015.0,0.0,0.0,1.0,0.0
21417,465750.0,3.0,2.50,2670.0,4534.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,0.0,-370.0,-545.0,2014.0,0.0,0.0,0.0,1.0


**Baseline Model**

In [9]:
base_model = df.copy()

X , y = base_model.drop(columns=['price']), base_model[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)


In [10]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

# train_preds = model1.predict(X_train)
# y_train == train_preds 

#test_preds = model1.predict(X_test)

cv_results = cross_val_score(model1, X_train, y_train, cv=5)
print(cv_results)
cv_results_mean = np.mean(cross_val_score(model1, X, y, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean)
#model1.score(y_test, test_preds)

[0.77129001 0.7394103  0.75512911 0.73825387 0.74757268]
-34298798477.552696


In [11]:
scaler = StandardScaler()
# Calculate the standard deviation of the X_train dataset
scaler.fit(X_train)
# convert all values into their standard deviation equivalents.
X_train_scaled = scaler.transform(X_train)

scaler.fit(y_train)
y_train_scaled = scaler.transform(y_train)


In [12]:
model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)

# train_preds = model_scaled.predict(X_train)
test_preds_scaled = model_scaled.predict(X_train_scaled)

cv_results_scaled = cross_val_score(model_scaled, X_train_scaled, y_train_scaled, cv=5)
print(cv_results_scaled)
cv_results_mean_scaled = np.mean(cross_val_score(model_scaled, X_train_scaled, y_train_scaled, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled)
#model_scaled.score(test_preds, y_test)

[0.77174087 0.73937868 0.75512693 0.73825336 0.74758818]
-0.2500540308608881


In [13]:
'''
model1 = LinearRegression()
model1.fit(X_train, y_train)

# train_preds = model1.predict(X_train)
# y_train == train_preds 

test_preds = model1.predict(X_test)

cv_results = cross_val_score(model1, X_train, y_train, cv=5)
cv_results

model1.score(y_test, test_preds)
'''

'\nmodel1 = LinearRegression()\nmodel1.fit(X_train, y_train)\n\n# train_preds = model1.predict(X_train)\n# y_train == train_preds \n\ntest_preds = model1.predict(X_test)\n\ncv_results = cross_val_score(model1, X_train, y_train, cv=5)\ncv_results\n\nmodel1.score(y_test, test_preds)\n'

**Logging the scaled target (y) with the scaled predictors** 

In [14]:
#df_price_log = pd.DataFrame([])
df_price_log = np.log(y_train) #['log_price']
 
scaler.fit(df_price_log)
y_scaled_log_price_train = scaler.transform(df_price_log)

In [15]:
model_scaled_log_price = LinearRegression()
model_scaled_log_price.fit(X_train_scaled, y_scaled_log_price_train)

# train_preds = model_scaled.predict(X_train)
#test_preds = model_scaled.predict(X_test_scaled)

cv_results = cross_val_score(model_scaled_log_price, X_train_scaled, y_scaled_log_price_train, cv=5)
print(cv_results)
cv_results_mean_scaled_ylog = np.mean(cross_val_score(model_scaled_log_price, X_train_scaled, y_scaled_log_price_train, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled_ylog)

[0.82101237 0.81409015 0.80113403 0.81274972 0.8114094 ]
-0.18783287413158262


In [16]:
#[0.82084467 0.81407879 0.80112027 0.81274158 0.81141183]
#-34224819193.55484

82101237

82101237

In [17]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Seattle,Snoqualmie,Woodinville,sqft_living_diff,sqft_lot_diff,year,x0_fall,x0_spring,x0_summer,x0_winter
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,-160.0,0.0,2014.0,1.0,0.0,0.0,0.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0.0,3.0,7.0,...,1.0,0.0,0.0,880.0,-397.0,2014.0,0.0,0.0,0.0,1.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0.0,3.0,6.0,...,0.0,0.0,0.0,-1950.0,1938.0,2015.0,0.0,0.0,0.0,1.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0.0,5.0,7.0,...,1.0,0.0,0.0,600.0,0.0,2014.0,0.0,0.0,0.0,1.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0.0,3.0,8.0,...,0.0,0.0,0.0,-120.0,577.0,2015.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21414,700000.0,3.0,2.50,2230.0,4006.0,2.0,0,0.0,3.0,8.0,...,1.0,0.0,0.0,0.0,-174.0,2015.0,0.0,1.0,0.0,0.0
21415,800000.0,4.0,3.25,3540.0,159430.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,0.0,1600.0,-232610.0,2014.0,0.0,0.0,0.0,1.0
21416,899000.0,4.0,3.50,2490.0,5500.0,2.0,0,0.0,3.0,9.0,...,1.0,0.0,0.0,780.0,0.0,2015.0,0.0,0.0,1.0,0.0
21417,465750.0,3.0,2.50,2670.0,4534.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,0.0,-370.0,-545.0,2014.0,0.0,0.0,0.0,1.0


**Poly Features**

In [18]:
#run on subset columns drop original after merge to main dataframe
pf = PolynomialFeatures(degree=3) #cubed


# Fitting the PolynomialFeatures object
pf.fit(X_train_scaled)

PolynomialFeatures(degree=3)

In [19]:
pdf = pd.DataFrame(pf.transform(X_train_scaled), columns=pf.get_feature_names())
pdf

,1,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x42^3,x42^2 x43,x42^2 x44,x42 x43^2,x42 x43 x44,x42 x44^2,x43^3,x43^2 x44,x43 x44^2,x44^3
0,1.0,2.906499,-0.145449,0.299277,-0.202071,0.944605,-0.082617,0.997435,-0.628659,0.298311,...,3.424450,-1.447788,-1.040014,0.612096,0.439697,0.315855,-0.258782,-0.185895,-0.133537,-0.095926
1,1.0,-0.406806,-0.472683,-0.074559,-0.241321,-0.916912,-0.082617,-0.307710,2.458631,-0.559677,...,-0.292018,0.690709,-0.201489,-1.633732,0.476581,-0.139025,3.864262,-1.127257,0.328836,-0.095926
2,1.0,0.697629,-0.472683,-0.327448,-0.085868,-0.916912,-0.082617,-0.307710,2.458631,1.156299,...,-0.292018,-0.280490,0.961526,-0.269417,0.923568,-3.166013,-0.258782,0.887109,-3.041031,10.424722
3,1.0,-0.406806,0.509019,0.783065,0.906860,0.944605,-0.082617,-0.307710,0.914986,1.156299,...,-0.292018,-0.280490,0.961526,-0.269417,0.923568,-3.166013,-0.258782,0.887109,-3.041031,10.424722
4,1.0,-1.511241,-0.472683,-0.921188,-0.260006,-0.916912,-0.082617,-0.307710,0.914986,-0.559677,...,-0.292018,0.690709,-0.201489,-1.633732,0.476581,-0.139025,3.864262,-1.127257,0.328836,-0.095926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15926,1.0,0.697629,0.509019,1.057944,-0.194145,0.944605,-0.082617,-0.307710,-0.628659,0.298311,...,-0.292018,0.690709,-0.201489,-1.633732,0.476581,-0.139025,3.864262,-1.127257,0.328836,-0.095926
15927,1.0,-1.511241,-1.454385,-1.459952,-0.213139,-0.916912,-0.082617,2.302579,-0.628659,-1.417665,...,3.424450,-1.447788,-1.040014,0.612096,0.439697,0.315855,-0.258782,-0.185895,-0.133537,-0.095926
15928,1.0,0.697629,0.509019,1.134910,-0.203856,0.944605,-0.082617,-0.307710,-0.628659,1.156299,...,-0.292018,-0.280490,0.961526,-0.269417,0.923568,-3.166013,-0.258782,0.887109,-3.041031,10.424722
15929,1.0,-2.615676,-1.454385,-1.492937,-0.178602,-0.916912,12.104101,4.912868,0.914986,-1.417665,...,3.424450,-1.447788,-1.040014,0.612096,0.439697,0.315855,-0.258782,-0.185895,-0.133537,-0.095926


In [20]:
#model_poly = LinearRegression()

#model_poly.fit(pdf, y_scaled_log_price_train)

#model_poly.score(pdf, y_scaled_log_price_train)



In [21]:
#cv_results_poly = cross_val_score(model_poly, pdf, y_scaled_log_price_train, cv=5)
#print(cv_results)
#cv_results_mean_scaled_ylog_poly = np.mean(cross_val_score(model_poly, pdf, y_scaled_log_price_train, cv=5,  scoring='neg_mean_squared_error'))
#print(cv_results_mean_scaled_ylog)

**Trying RFE**

In [22]:
X_new = df.drop(columns=['price'])
scaler_new = StandardScaler()
X_new = scaler.fit_transform(X)
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=25)
select.fit(X=X_new, y=df['price'])
#list(zip(df.columns, select.support_))

RFE(estimator=LinearRegression(), n_features_to_select=25)

In [23]:
cv_results = cross_val_score(lr_rfe, X_train_scaled, y_scaled_log_price_train, cv=5)
print(cv_results)
cv_results_mean_scaled_ylog = np.mean(cross_val_score(lr_rfe, X_train_scaled, y_scaled_log_price_train, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled_ylog)

[0.82101237 0.81409015 0.80113403 0.81274972 0.8114094 ]
-0.18783287413158262


, 'sqft_lot15', 'sqft_living15', 'Auburn', 'Bellevue', 'Black_diamond', 
                                 'Bothell', 'Duball', 'Enumclaw', 'Fall city', 'Federal_way', 'Issaquah', 'Kenmore',
                                 'Kent', 'Kirkland', 'Maple Valley', 'Medina', 'Mercer Island', 'North Bend', 
                                 'Renton', 'Sammamish','sqft_lot_diff', 'year', 'x0_fall',
                                 'x0_spring'

In [24]:
df.iloc[:,:39].corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Kirkland,Maple Valley,Medina,Mercer Island,North Bend,Renton,Sammamish,Seattle,Snoqualmie,Woodinville
price,1.000000,0.315489,0.523554,0.700111,0.087857,0.257886,0.263293,0.394071,0.036223,0.665936,...,0.062069,-0.079563,0.214753,0.206246,-0.028401,-0.105859,0.101871,0.020874,-0.003567,0.031440
bedrooms,0.315489,1.000000,0.529223,0.593188,0.033279,0.187680,-0.003146,0.080454,0.023311,0.367529,...,0.031222,0.012773,0.037491,0.084177,-0.014400,0.039889,0.077080,-0.185116,0.024902,0.024449
bathrooms,0.523554,0.529223,1.000000,0.755940,0.088031,0.503316,0.062969,0.186105,-0.126398,0.664513,...,0.027658,0.057235,0.069507,0.090638,0.001252,0.004928,0.124461,-0.208438,0.068436,0.043676
sqft_living,0.700111,0.593188,0.755940,1.000000,0.172119,0.358539,0.104336,0.282074,-0.060347,0.762367,...,0.016372,0.012487,0.092162,0.129548,-0.001310,0.004670,0.154845,-0.218916,0.065819,0.080648
sqft_lot,0.087857,0.033279,0.088031,0.172119,1.000000,-0.003450,0.021483,0.075554,-0.009976,0.113437,...,-0.026867,0.041713,0.002963,-0.004074,0.084858,-0.007860,0.006314,-0.138971,0.006460,0.092889
floors,0.257886,0.187680,0.503316,0.358539,-0.003450,1.000000,0.021157,0.028848,-0.263395,0.459227,...,-0.039381,0.077803,0.006378,0.002812,0.002230,-0.013661,0.096842,-0.036866,0.086419,0.014891
waterfront,0.263293,-0.003146,0.062969,0.104336,0.021483,0.021157,1.000000,0.381187,0.016636,0.082979,...,0.003776,-0.013927,0.008096,0.045448,-0.008422,-0.016785,0.026190,0.024728,-0.009923,-0.012443
view,0.394071,0.080454,0.186105,0.282074,0.075554,0.028848,0.381187,1.000000,0.044459,0.249708,...,0.003370,-0.045401,0.013893,0.080685,0.004945,-0.056812,0.002231,0.069430,0.010216,-0.038407
condition,0.036223,0.023311,-0.126398,-0.060347,-0.009976,-0.263395,0.016636,0.044459,1.000000,-0.148139,...,0.005308,-0.056789,0.004754,0.066808,-0.039417,0.032443,-0.074323,0.016031,-0.067494,-0.020079
grade,0.665936,0.367529,0.664513,0.762367,0.113437,0.459227,0.082979,0.249708,-0.148139,1.000000,...,0.017379,-0.008560,0.078373,0.128725,-0.002988,-0.030292,0.197421,-0.183784,0.021156,0.091210


**Trying a Robust Scaler**

In [25]:
transformer = RobustScaler()
# Calculate the standard deviation of the X_train dataset
transformer.fit(X_train)
# convert all values into their standard deviation equivalents.
X_train_robust_scaled = transformer.transform(X_train)

#X_test_robust_scaled = transformer.transform(X_test)

# transformer.fit(x_train)
# y_train_robust_scaled = transformer.transform(y_train)


In [26]:
model_robust_scaled = LinearRegression()
model_robust_scaled.fit(X_train_robust_scaled, y_scaled_log_price_train)

cv_results_scaled = cross_val_score(model_robust_scaled, X_train_robust_scaled, y_scaled_log_price_train, cv=5, scoring='r2')
print(cv_results_scaled)
cv_results_mean_scaled = np.mean(cross_val_score(model_robust_scaled, X_train_robust_scaled, y_scaled_log_price_train, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled)
#model_scaled.score(test_preds, y_test)

[0.82097712 0.81408318 0.80090049 0.81242719 0.811414  ]
-0.18795118794983204


[0.82097558 0.81408642 0.8011188  0.8127386  0.81141413]
-0.18784531187732628

**Getting score with final model on test**

In [28]:
#model_robust_scaled.score(X_test_robust_scaled, y_test)

In [29]:
model_robust_scaled.score(X_test, y_test)

-2.5177438360128464e+17

**Trying a unit vector transformation**

In [32]:

unit_vector_X = X_train / np.linalg.norm(X_train)
#unit_vector

unit_vector_y = y_train / np.linalg.norm(y_scaled_log_price_train)

unit_vector_lr = LinearRegression()
model_robust_scaled.fit(unit_vector_X, unit_vector_y)

cv_results_scaled = cross_val_score(unit_vector_lr, unit_vector_X, y_scaled_log_price_train, cv=5, scoring='r2')
print(cv_results_scaled)
cv_results_mean_scaled = np.mean(cross_val_score(unit_vector_lr, unit_vector_X, y_scaled_log_price_train, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled)

[0.82097689 0.81408088 0.80112438 0.81273711 0.80932964]
-0.18825761885858355


**trying to find out how to scale different columns**

In [33]:
#plt.loglog(X_train,y_train,label='X vs Y')

**Feature engineering some new columns and removing some**

In [34]:
df_new = df.copy()

In [35]:
df_new['yr_till_renovation'] = df_new['yr_renovated'] - df_new['yr_built']
df_new['sqft_living_above'] = df_new['sqft_above'] - df_new['sqft_living']
df_new['yr_since_built'] = df_new['year'] - df_new['yr_built']
to_drop = ['sqft_lot_diff', 'sqft_living15', 'yr_renovated', 'basement', 'year']
df_new.drop(to_drop, axis=1, inplace=True)

In [44]:
df_new

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Snoqualmie,Woodinville,sqft_living_diff,x0_fall,x0_spring,x0_summer,x0_winter,yr_till_renovation,sqft_living_above,yr_since_built
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0.0,3.0,7.0,...,0.0,0.0,-160.0,1.0,0.0,0.0,0.0,0.0,0.0,59.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0.0,3.0,7.0,...,0.0,0.0,880.0,0.0,0.0,0.0,1.0,40.0,-400.0,63.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0.0,3.0,6.0,...,0.0,0.0,-1950.0,0.0,0.0,0.0,1.0,0.0,0.0,82.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0.0,5.0,7.0,...,0.0,0.0,600.0,0.0,0.0,0.0,1.0,0.0,-910.0,49.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0.0,3.0,8.0,...,0.0,0.0,-120.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21414,700000.0,3.0,2.50,2230.0,4006.0,2.0,0,0.0,3.0,8.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
21415,800000.0,4.0,3.25,3540.0,159430.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,1600.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0
21416,899000.0,4.0,3.50,2490.0,5500.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,780.0,0.0,0.0,1.0,0.0,0.0,-710.0,0.0
21417,465750.0,3.0,2.50,2670.0,4534.0,2.0,0,0.0,3.0,9.0,...,0.0,0.0,-370.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0


In [38]:


X_2 , y_2 = df_new.drop(columns=['price']), df_new[['price']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=8)

In [39]:
new_model = LinearRegression()
new_model.fit(X_train_2, y_train_2)

cv_results = cross_val_score(model1, X_train, y_train, cv=5)
print(cv_results)
cv_results_mean = np.mean(cross_val_score(model1, X, y, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean)

[0.77129001 0.7394103  0.75512911 0.73825387 0.74757268]
-34298798477.552696


In [40]:
scaler = StandardScaler()
# Calculate the standard deviation of the X_train dataset
scaler.fit(X_train_2)
# convert all values into their standard deviation equivalents.
X_train_scaled_2 = scaler.transform(X_train_2)

scaler.fit(y_train_2)
y_train_scaled_2 = scaler.transform(y_train_2)

In [45]:
model_scaled_2 = LinearRegression()
model_scaled_2.fit(X_train_scaled_2, y_train_2)

# train_preds = model_scaled.predict(X_train)
#test_preds_scaled = model_scaled.predict(X_train_scaled_2)

cv_results_scaled = cross_val_score(model_scaled_2, X_train_scaled_2, y_train_scaled_2, cv=5)
print(cv_results_scaled)
cv_results_mean_scaled = np.mean(cross_val_score(model_scaled_2, X_train_scaled_2, y_train_scaled_2, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled)

[0.77095064 0.74001186 0.75417972 0.7380603  0.74739925]
-0.25031992268302383


In [42]:
#y
df_price_log_2 = np.log(y_train_2) #['log_price']
#X
df_bedrooms_log = np.log(y_train_2)
df_bedrooms_log = np.log(y_train_2)
df_bedrooms_log = np.log(y_train_2)
df_bedrooms_log = np.log(y_train_2)
df_bedrooms_log = np.log(y_train_2)
df_bedrooms_log = np.log(y_train_2)


scaler.fit(df_price_log_2)
y_scaled_log_price_train_2 = scaler.transform(df_price_log_2)

In [43]:
model_scaled_log_price_2 = LinearRegression()
model_scaled_log_price_2.fit(X_train_scaled_2, y_scaled_log_price_train_2)

# train_preds = model_scaled.predict(X_train)
#test_preds = model_scaled.predict(X_test_scaled)

cv_results = cross_val_score(model_scaled_log_price_2, X_train_scaled_2, y_scaled_log_price_train_2, cv=5)
print(cv_results)
cv_results_mean_scaled_ylog = np.mean(cross_val_score(model_scaled_log_price_2, X_train_scaled_2, y_scaled_log_price_train_2, cv=5,  scoring='neg_mean_squared_error'))
print(cv_results_mean_scaled_ylog)

[0.81890426 0.81196345 0.79928995 0.81123302 0.80944272]
-0.18974567353899416
